# Introduction

This notebook will pull a Spotify playlist's data using Spotipy libraries and Spotify API.

For more information, please visit https://spotipy.readthedocs.io/en/latest/. Watch https://www.youtube.com/watch?v=vipVEWe86Lg&t=242s for more information

The required inputs for this notebooks are as followings: 
- Spotify Client ID and Secret key, which can be found in Spotify Dashboard
- User's Spotify ID
- Playlist

Import Libraries

In [ ]:
import urllib.request
from urllib.request import urlopen

import sys
import spotipy
import spotipy.util as util
import os
import json
import webbrowser
from json.decoder import JSONDecodeError

Expand Screen

In [117]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Input

In [128]:
# My Spotify ID: https://open.spotify.com/user/1214868237
user_spotify_id = '1214868237'
user_spotify_name = 'anhquocduong611'

USERNAME = sys.argv[1] 
CLIENT_ID = '6efd3e7d946d45b2ae1c1cd274dd8a51'# Go to Spotify App Developer Dashboard
CLIENT_SECRET = 'a424e3ab68364de18e90295467dafb7c' # Go to Spotify App Developer Dashboard
REDIRECT_URI = 'http://google.com/' # Set it as 'http://google.com/'
SCOPE = 'user-library-read' # will give user to modify

In [119]:
playlist_name = 'Anjunadeep'

# Script Body

###### Get access using API

In [120]:
try:
    token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)
except:
    os.remove(f".cache~{username}")
    token = util.prompt_for_user_token(username = USERNAME, 
                                   scope = SCOPE, 
                                   client_id = CLIENT_ID, 
                                   client_secret = CLIENT_SECRET, 
                                   redirect_uri = REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

### Get User's Playlist

In [121]:
# Funciton to show track
def show_tracks(tracks):
    track_list = []
    for i, item in enumerate(tracks['items']):
        track = item['track']
        track_list.append((i, track['artists'][0]['name'], track['name'], track['album']['name']))
    return track_list

# Get playlist from Spotify API
playlists = sp.user_playlists(user_spotify_id)

for playlist in playlists['items']:
    if playlist['owner']['id'] == user_spotify_id and playlist['name'] ==playlist_name:
        print(playlist['name'])
        print('Total tracks', playlist['tracks']['total'])
        results = sp.user_playlist(user_spotify_id, playlist['id'], fields="tracks,next")
        tracks = results['tracks']
        my_playlist = show_tracks(tracks)
        while tracks['next']:
            tracks = sp.next(tracks)
            my_playlist = show_tracks(tracks)
        print(my_playlist)

Anjunadeep
Total tracks 35
[(0, 'Eli & Fur', 'Night Blooming Jasmine - Rodriguez Jr. Remix Edit', 'Night Blooming Jasmine (Rodriguez Jr. Remix)'), (1, 'Yotto', 'North', 'North'), (2, 'Gabriel & Dresden', 'Sequoia - Edit', 'The Only Road'), (3, 'Jan Blomqvist', 'The Space In Between', 'Disconnected'), (4, 'Solomun', "Somebody's Story", 'Black Label #36'), (5, 'Einmusik', 'Tainted', 'Pristine'), (6, 'Worakls', 'Nocturne', 'Hungry 5 (The Best of 5 Years)'), (7, 'Solomun', 'The Way Back', 'Black Label #36'), (8, 'Claptone', 'Heartbeat (feat. Nathan Nicholson)', 'Charmer'), (9, 'Dominik Eulberg', 'Dream Machine', 'Dream Machine'), (10, 'Worakls', 'Toi', 'Hungry 5 (The Best of 5 Years)'), (11, 'Luttrell', 'Contact', 'Anjunadeep 09'), (12, 'Gallago', 'Lost Signal', 'Anjunadeep 09'), (13, 'SOHN', 'Lights - SQL Remix', 'Anjunadeep 09'), (14, 'Gacha Bakradze', 'Contactless', 'Anjunadeep 09'), (15, 'Modd', 'Abenaki', 'Anjunadeep 09'), (16, 'Jody Wisternoff', 'For All Time', 'Anjunadeep 09'), (17,

Format the track name

In [122]:
for i in range(len(my_playlist)):
    my_playlist[i] = list(my_playlist[i])

# Remove character ' in the track data
for i in range(len(my_playlist)):
    for y in range(1,3):
        my_playlist[i][y] = my_playlist[i][y].replace("'","")

### Get Track's ID

In [123]:
def search_track_id(track_name, artist, album):
    # Create a dictionary for result
    search_result = dict.fromkeys(['track_name', 'artist', 'album', 'track_id', 'popularity'])
    
    # Limit number of raw result output of 10 and market in US
    # Search by Track name first
    if album == '':
        album = '{}'
    else: 
        album = album
       
    searched_raw_result = sp.search(q="album: "+ album + " artist: " + artist + " track: " + track_name, limit=1, type='track', market=None)
    # Search by Artist name
    searched_raw_result1 = searched_raw_result['tracks']['items']
    
    if len(searched_raw_result1) >0:
        search_result.update(track_name = searched_raw_result1[0]['name'],
                             artist = searched_raw_result1[0]['album']['artists'][0]['name'], 
                             album = searched_raw_result1[0]['album']['name'],
                             track_id = searched_raw_result1[0]['uri'][14:],
                             popularity = searched_raw_result1[0]['popularity'])
    return search_result

### Get Track Data from User's Playlist

In [124]:
playlist_data = []
for i in range(len(my_playlist)):
    # Get Track ID by track name, artist and album collected from the playlist
    track_find_id = search_track_id(track_name = my_playlist[i][2], artist = my_playlist[i][1], album = my_playlist[i][3])

    # Get Track Audio Features
    track_audio_features = sp.audio_features(track_find_id['track_id'])
    
    # Get Track Audio Analysis
    audio_analysis_dict = dict.fromkeys(['sections'])
    audio_analysis_dict.update(sections = sp.audio_analysis(track_find_id['track_id'])['sections'])

    # Merge 2 dictionaries into one
    track_audio_data = dict(track_audio_features[0], **audio_analysis_dict)
    track_data = dict(track_find_id, **track_audio_data)
    
    playlist_data.append(track_data)

###### Sample

In [125]:
print(playlist_data[0].keys())

dict_keys(['track_name', 'artist', 'album', 'track_id', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'sections'])


In [130]:
import json
json = json.dumps(playlist_data)

f = open("C:/Users/anhqu/OneDrive/Desktop/QAs/QA github folder/" +  user_spotify_name + '_' + playlist_name + "_playlist_data.json","w")
f.write(json)
f.close()